# 1. Data Preprocessing


## Objective

The data used during this project has been obtained from a historical database of hourly sensor readings in Tlaquepaque, and although it's very useful information, it has some inconsistencies. The main problem this database has is incomplete data and a lack of standard practices in creating the datasets (various NULL representatives, missing features, etc.).

Before performing any forecasting operations on the data, it will be preprocessed. Some of the changes made will be particular to the handling of time series data. 

These changes will mostly be comprised of: 

- Standarizing column names.
- Extract relevant features.
- Identify null values.
- Determining feature data types.
- Reducing number of instances.
- Reframing the dataset to have a time series format.

In [16]:
import pandas as pd
import numpy as np

## Standarize column names and extract relevant features

After conducting an analysis of data from years between 2008 and 2023, the yearly readings of 2017, 2020 and 2023 have been specifically chosen because of the integrity of their data.



### Description of relevant features

Forecasting the readings for the pollutants in Tlaquepaque requires the following features:

_Exogenous variables:_

- Date (Fecha)
- Hour (Hora)
- Station (Estación)
- Temperature (TMP)
- Relative Humidity (RH)
- Barometric Pressure (PBA)
- Wind Speed (WS)
- Wind Direction (WD)
- Precipitation (PP)

_Pollutants:_

- Fine Particulate Matter less than 2.5 micrometers (PM2.5)
- O3 (Ozone)
- NO (Nitrogen Oxide)
- NO2 (Nitrogen Dioxide)
- NOX (Nitrogen Oxides)
- CO (Carbon Monoxide)
- PM10 (Fine Particulate Matter less than 10 micrometers)

This project aims to forecast the PM2.5 readings for Tlaquepaque, but in case this projects is expanded to forecast all other pollutants, the dataset will be preprocessed and interpolated for all pollutants.


All other features can be ignored. These are:

_TMPI (Internal Temperature), Solar Radiation (RS), and UVI (UV Index)._

In [17]:
# Read the csv file
filename = "semadet-aire-2023"
filepath = f"datasets/{filename}.csv"

df = pd.read_csv(filepath, encoding='utf-8')

# Normalize column names
df.columns = df.columns.str.lower().str.strip()

# Rename certain columns 
df.rename({"pm2.5": "pm25", 
           "date_time": "date",
           "precipitacion": "pp", 
           "rad solar": "rs",
           "presion barometrica": "pba"},
          axis="columns",
          inplace=True)

# irrelevant_features = ["rs", "nox", "no"] # Irrelevant features 2017
irrelevant_features = ["rs", "nox", "no", "tmpi", "uvi"] # Irrelevant features 2023
df.drop(irrelevant_features, axis="columns", inplace=True)


In [18]:
# See first three rows of data
df.head(3)

,estacion,date,hora,o3,no2,so2,co,pm10,pm25,tmp,rh,ws,wd,pp,pba
0,Aguilas,1/1/2023,0,0.002,SE,SE,SE,61.8,58.1,12.6,88.7,0.38,190.77,0.25,SE
1,Aguilas,1/1/2023,1,0.002,SE,SE,SE,83.8,76.5,12.1,89.8,1.27,215.13,0,SE
2,Aguilas,1/1/2023,2,0.003,SE,SE,SE,98.2,95,11.8,89.5,2.44,240.82,0,SE


## Identify null values

There is a variety of values used to identify for null data. All the identifiers for the null values will be standarized to see which columns to keep based on amount of values.

In [19]:
def replace_with_null(row):
    # Null value identifiers
    null_values = ["IO", "SE", "ND", "IF", "VE", "IR", "VZ", "IC", "IR 1000",
                   "IR valor 1000", " ", "", "-", "SD"]
    
    # Columns that don't have null values
    exclude_columns = ["estacion", "date", "hora"]
    
    # Replace null values by feature (column)
    for column in row.index:
        if column not in exclude_columns and row[column] in null_values:
            row[column] = np.nan
            
    return row
    

In [20]:
# Replace null values for each column in dataframe
df = df.apply(replace_with_null, axis="columns")

Looking at the Non-Null count for each feature, NO2 and SO2 have no useful data so they'll be dropped entirely.

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87600 entries, 0 to 87599
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   estacion  87600 non-null  object 
 1   date      87600 non-null  object 
 2   hora      87600 non-null  int64  
 3   o3        49551 non-null  object 
 4   no2       0 non-null      float64
 5   so2       0 non-null      float64
 6   co        28843 non-null  object 
 7   pm10      57768 non-null  object 
 8   pm25      50068 non-null  object 
 9   tmp       44625 non-null  object 
 10  rh        46416 non-null  object 
 11  ws        49970 non-null  object 
 12  wd        39858 non-null  object 
 13  pp        38714 non-null  object 
 14  pba       39048 non-null  object 
dtypes: float64(2), int64(1), object(12)
memory usage: 10.0+ MB


In [22]:
# Drop unuseful columns
df.drop(["no2", "so2"], axis="columns", inplace=True)

## Specify data tyoes

Now that the null values have been correctly identified, every feature that represents a pollutant or exogenous variable can be correctly cast as a float.

In [23]:
# Columns that should be treated as floats
float_cols = ["o3", "co", "pm10", "pm25", "tmp", "rh", "ws", "wd", "pp"]

# Cast columns to floats
df[float_cols] = df[float_cols].astype('float')

## Extract relevant instances

The yearly dataset provides hourly readings for many regions in Guadalajara. As this project only aims to forecast PM2.5 for Tlaquepaque, all the instances that belong to this region will be extracted, making the feature Station (Estacion) no longer useful. 

In [24]:
# Extract instances that belong to the target region
df = df[df["estacion"] == "Tlaquepaque"]

In [25]:
# Drop unuseful column
df.drop("estacion", axis="columns", inplace=True)

## Reframe dataset as timeseries

In order to treat the data as a timeseries, the "Date" column will be used as the index.

In [26]:
# Use date as index
df.index = pd.to_datetime(df['date'], format='%m/%d/%Y')
# Drop the date column
df.drop("date", axis="columns", inplace=True)

Now that the dates are the index, the instances will be sorted in ascending order to preserve seasonal trends and patterns.

In [27]:
# Sort dates 
df = df.sort_index()

## Reduce number of instances

The objective of this project is to forecast the daily PM2.5 readings for Tlaquepaque. Only one entry per day will be preserved by calculating the average of all the readings that belong to the same day.

In the special case of Wind Direction, the fact that the measurement is taken in degrees will be taken into account by calculating the cirular mean.

In [28]:
def circular_mean(angles):
    angles_rad = np.deg2rad(angles)  
    mean_sin = np.mean(np.sin(angles_rad))
    mean_cos = np.mean(np.cos(angles_rad))
    mean_angle = np.arctan2(mean_sin, mean_cos)  
    return np.rad2deg(mean_angle) % 360 

In [29]:
# Specify average as the aggregating function every feature except wind direction and hour
aggregation_functions = {col: "mean" for col in df.columns if col not in ["wd", "hora"]}

# Specify circular mean for Wind Direction
aggregation_functions["wd"] = circular_mean

In [30]:
# Group all instances by date and apply average functions
df_daily = df.groupby("date").agg(aggregation_functions)

After reducing the number of instances and analyzing the Non-Null count by column, PBA and CO have no relevant information at all, so they'll be eliminated.

It can also be seen that now there are maximum 365 instances per feature (one for every day of the year.)

In [31]:
df_daily.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 365 entries, 2023-01-01 to 2023-12-31
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   o3      180 non-null    float64
 1   co      0 non-null      float64
 2   pm10    365 non-null    float64
 3   pm25    365 non-null    float64
 4   tmp     131 non-null    float64
 5   rh      315 non-null    float64
 6   ws      319 non-null    float64
 7   pp      365 non-null    float64
 8   pba     0 non-null      object 
 9   wd      257 non-null    float64
dtypes: float64(9), object(1)
memory usage: 31.4+ KB


In [32]:
# Drop unuseful columns
df_daily.drop(["pba", "co"], axis="columns", inplace=True)

## Save pre processed data

Once the data has been properly processed, it'll be saved as its own file. This process has been repeated for each yearly dataset that's been found useful.

In [33]:
# df_daily.to_csv(f"datasets/preprocess/{filename}-processed.csv")